In [122]:
import pandas as pd

# Loading the spam data
# ham is the label for non-spam messages
spam = pd.read_excel('test.xlsx')
spam.head(10)

,Status,Decisão
0,parcialmente provido,conhecido. parcialmente provido. unanime
1,desprovido,conhecido. improvido. unânime
2,parcialmente provido,conhecido. parcialmente provido. unânime
3,desprovido,conhecer em parte. negar provimento. unânime
4,desprovido,conhecer do recurso e negar-lhe provimento. un...
5,provido,conhecido. provido. unânime.
6,desprovido,apelação da autora e recurso adesivo da ré con...
7,desprovido,conhecido. preliminar rejeitada. prejudicial r...
8,parcialmente provido,dar parcial provimento ao recurso de m.c.b.s.r...
9,desprovido,conhecer do recurso e negar-lhe provimento.


In [123]:
import spacy
nlp = spacy.blank("pt")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

In [124]:
# Add labels to text classifier
textcat.add_label("provido")
textcat.add_label("desprovido")
textcat.add_label("parcialmente provido")
textcat.add_label("parcialmente improvido")

1

In [125]:
train_texts = spam['Decisão'].values
train_labels = [{'cats': {'provido': label == 'provido',
                          'desprovido': label == 'desprovido',
                          'parcialmente provido': label == 'parcialmente provido',
                          'parcialmente improvido': label == 'parcialmente improvido',
                         }
                } 
                for label in spam['Status']]

In [126]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('conhecido. parcialmente provido. unanime',
  {'cats': {'provido': False,
    'desprovido': False,
    'parcialmente provido': True,
    'parcialmente improvido': False}}),
 ('conhecido. improvido. unânime',
  {'cats': {'provido': False,
    'desprovido': True,
    'parcialmente provido': False,
    'parcialmente improvido': False}}),
 ('conhecido. parcialmente provido. unânime',
  {'cats': {'provido': False,
    'desprovido': False,
    'parcialmente provido': True,
    'parcialmente improvido': False}})]

In [127]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) but we need to
    # send separate lists for texts and labels to update().
    # This is a quick way to split a list of tuples into lists
    texts, labels = zip(*batch)
    nlp.update(texts, labels, sgd=optimizer)

In [140]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(20):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 0.6548962322995067}
{'textcat': 1.286041172221303}
{'textcat': 1.8963963678106666}
{'textcat': 2.4987501250579953}
{'textcat': 3.1268228692933917}
{'textcat': 3.8827230846509337}
{'textcat': 4.3834200175479054}
{'textcat': 5.002194302622229}
{'textcat': 5.44556794827804}
{'textcat': 5.785625381860882}
{'textcat': 6.3442628756165504}
{'textcat': 6.831632277928293}
{'textcat': 7.365547354333103}
{'textcat': 7.885119690559804}
{'textcat': 8.348310220986605}
{'textcat': 8.886866567190737}
{'textcat': 8.969448438379914}
{'textcat': 9.171361861284822}
{'textcat': 9.762848474085331}
{'textcat': 10.263680576812476}


In [141]:
texts = ["conhecer do recurso e negar-lhe provimento. unânime.", 
         "conhecer. dar parcial improvimento. unânime",
         "apelação parcialmente conhecida e parcialmente provida. unânime.",
         "prover-lhe provimento"]
docs = [nlp.tokenizer(text) for text in texts]   
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[0.19849427 0.63169545 0.12702602 0.04278427]
 [0.32756126 0.23919512 0.17616656 0.2570771 ]
 [0.11642125 0.20333734 0.5724111  0.10783035]
 [0.35534903 0.28463045 0.19674794 0.16327254]]


In [142]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['desprovido', 'provido', 'parcialmente provido', 'provido']


In [143]:
df = pd.read_excel('Dados.xlsx', index=False, usecols = [7])  
my_texts = []
for i in df.values.tolist():
    my_texts.append(str(i[0]).lower())
    
print(len(my_texts))

6694


In [144]:
my_texts = my_texts[:25]
print(my_texts)
docs = [nlp.tokenizer(a) for a in my_texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

['negou-se provimento aos recursos. unânime', 'conhecer. rejeitar preliminar. negar provimento.unânime', 'negar provimento. unânime', 'negar provimento. unânime', 'conhecer e dar parcial provimento, unânime', 'conhecido. negou-se provimento. unânime', 'conhecido. negou-se provimento. unânime', 'conhecido. negou-se provimento. unânime', 'conhecer e negar provimento, unânime', 'conhecer e negar provimento, unânime', 'acolher a preliminar de ilegitimidade passiva dosegundo réu para julgar extinto o processo em relação a ele.dar parcial provimento ao recurso da primeira ré. unânime', 'conhecer e negar provimento, unânime', 'conhecer e dar parcial provimento, unânime', 'dar parcial provimento ao recurso do autor. darparcial provimento ao recurso dos réus. unânime', 'conhecer. negar provimento. unânime', 'conhecido. desprovido. unânime', 'conhecer do apelo da 2ª ré e negar provimento.conhecer em parte do apelo do autor e dar parcial provimento.unânime', 'conhecido. desprovido. unânime', 'con

In [145]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'provido', 'desprovido', 'desprovido', 'desprovido', 'desprovido', 'provido', 'provido', 'desprovido', 'provido', 'provido', 'parcialmente provido', 'desprovido']
